In [1]:
import json
import pickle
import numpy as np
import pandas as pd

In [2]:
method = 'pool'
embedding_type = 'roberta' # roberta, glove300, fastText

dataset_name = 'wos46985'
base_dir = '../../data/WOS/'
data_file = base_dir+'Meta-data/Data.csv'

text_embedding_dir = '../data/'+dataset_name
text_embedding_file = text_embedding_dir+'/'+embedding_type+'-embedding-'+method+'.pkl'

seed_indices_file = text_embedding_dir+'/seed_indices.json'
remaining_indices_file = text_embedding_dir+'/remaining_indices.json'

seed_embeddings_file = text_embedding_dir+'/cluster_hierarchy_seed_embeddings-roberta-'+method+'.pkl'

In [3]:
text_embeddings = pickle.load(open(text_embedding_file, "rb"))
print(text_embeddings.shape)

with open(seed_indices_file, "r") as infile:
    seeds = json.load(infile)

(46985, 768)


In [4]:
def get_topic_seeds(seedTopics, topic_seeds={}, main_topic_update_method=1, main_topic_weight=3):
    # Function to retrieve BERT embeddings from memory
    def get_embedding_text(index):
        topic_embedding = text_embeddings[index]
        return topic_embedding

    if include_sub_topics == True:
        main_topics = list(seedTopics.keys())
        topics = list(seedTopics.keys())

        for topic in topics:
            if topic not in topic_seeds.keys():
                indices = seedTopics[topic]['indices']
                topic_embedding = np.mean([get_embedding_text(ind) for ind in indices], axis=0)
                topic_seeds[topic] = topic_embedding

            try:
                for sub_topic in list(seedTopics[topic]['sub-topic'].keys()):
                    if topic+'/'+sub_topic not in topic_seeds.keys():
                        indices = seedTopics[topic]['sub-topic'][sub_topic]['indices']
                        topic_embedding = np.mean([get_embedding_text(ind) for ind in indices], axis=0)
                        topic_seeds[topic+'/'+sub_topic] = topic_embedding

                # Update topic embedding by taking mean of itself with all sub-topic embeddings
                if main_topic_update_method == 1:
                    topic_seeds[topic] = np.mean(
                        [
                          topic_seeds[topic+'/'+sub_topic] for sub_topic in list(seedTopics[topic]['sub-topic'].keys())
                        ] + [topic_seeds[topic]]*main_topic_weight
                        , axis=0
                    )
                elif main_topic_update_method == 2:
                    topic_seeds[topic] = np.mean(
                        [
                            np.mean(
                                [
                                  topic_seeds[topic+'/'+sub_topic] for sub_topic in list(seedTopics[topic]['sub-topic'].keys())
                                ]
                                , axis=0
                            )
                        ]
                        + [topic_seeds[topic]]*main_topic_weight
                        , axis=0
                    )
                elif main_topic_update_method == 3:
                    topic_seeds[topic] = np.mean(
                        [
                            np.mean(
                                [
                                  topic_seeds[topic+'/'+sub_topic] for sub_topic in list(seedTopics[topic]['sub-topic'].keys())
                                ]
                                , axis=0
                            )
                        ]
                        + [topic_seeds[topic]]*main_topic_weight*len(list(seedTopics[topic]['sub-topic'].keys()))
                        , axis=0
                    )
                else:
                    print('Not Altering Main Topic')
            except:
                print(topic,'Whoohs!')
                pass
    else:
        topics = list(seedTopics.keys())

        for topic in topics:
            if topic not in topic_seeds.keys():
                indices = seedTopics[topic]['indices']
                topic_embedding = np.mean([get_embedding_text(ind) for ind in indices], axis=0)
                topic_seeds[topic]= topic_embedding
    
    return topic_seeds

In [5]:
include_sub_topics = True
method = 'pool'

In [6]:
main_topic_update_method=2
main_topic_weight=2

In [7]:
topic_seeds = get_topic_seeds(
    seeds, {}, main_topic_update_method=main_topic_update_method, main_topic_weight=main_topic_weight
)

print(len(list(topic_seeds.keys())),'seeds:',list(topic_seeds.keys()),'\n')

140 seeds: ['0', '0/0', '0/1', '0/2', '0/3', '0/4', '0/5', '0/6', '0/7', '0/8', '0/9', '0/10', '0/11', '0/12', '0/13', '0/14', '0/15', '0/16', '1', '1/0', '1/1', '1/2', '1/3', '1/4', '1/5', '1/6', '1/7', '1/9', '1/10', '1/11', '1/12', '1/13', '1/14', '1/15', '2', '2/0', '2/1', '2/2', '2/3', '2/4', '2/5', '2/6', '2/7', '2/8', '2/9', '2/10', '2/11', '2/12', '2/13', '2/14', '2/15', '2/16', '2/17', '2/18', '3', '3/0', '3/1', '3/2', '3/3', '3/4', '3/5', '3/6', '3/7', '3/8', '4', '4/0', '4/1', '4/2', '4/3', '4/4', '4/5', '4/6', '4/7', '4/8', '4/9', '4/10', '5', '5/0', '5/1', '5/2', '5/3', '5/4', '5/5', '5/6', '5/7', '5/8', '5/9', '5/10', '5/11', '5/12', '5/13', '5/14', '5/15', '5/16', '5/17', '5/18', '5/19', '5/20', '5/21', '5/22', '5/23', '5/24', '5/25', '5/26', '5/27', '5/28', '5/29', '5/30', '5/31', '5/32', '5/33', '5/34', '5/35', '5/36', '5/37', '5/38', '5/39', '5/40', '5/41', '5/42', '5/43', '5/44', '5/45', '5/46', '5/47', '5/48', '5/49', '5/50', '5/51', '5/52', '6', '6/0', '6/1', '6/2'

In [8]:
pickle.dump(topic_seeds, open(seed_embeddings_file, "wb"))